# Árbol de Decisión de Regresión — Dataset Sintético

Este cuaderno genera un dataset sintético con **no linealidades**, **outliers** y **faltantes**, para entrenar y evaluar un árbol de decisión de regresión.

Incluye:
- Generación y **limpieza** (capado IQR + imputación de medianas)
- División train/test
- Modelo base y **tuning** con `GridSearchCV`
- **Curva de aprendizaje**, **importancias** y **visualización del árbol** (truncado)

> Nota: Se usa `matplotlib` sin estilos predefinidos y una figura por gráfico.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
import math
np.random.seed(42)


## Funciones auxiliares

In [ ]:
def cap_outliers_iqr(df, cols):
    capped = df.copy()
    for c in cols:
        q1 = capped[c].quantile(0.25)
        q3 = capped[c].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        capped[c] = np.clip(capped[c], lower, upper)
    return capped

def report_metrics(y_true, y_pred, label=""):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"{label}MAE:  {mae:.4f}")
    print(f"{label}MSE:  {mse:.4f}")
    print(f"{label}RMSE: {rmse:.4f}")
    print(f"{label}R²:   {r2:.4f}")

def plot_learning_curve(estimator, X, y, title="Curva de aprendizaje"):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, scoring="neg_mean_squared_error", n_jobs=None,
        train_sizes=np.linspace(0.1, 1.0, 5), shuffle=True, random_state=42
    )
    train_rmse = np.sqrt(-train_scores)
    test_rmse = np.sqrt(-test_scores)
    plt.figure()
    plt.title(title)
    plt.xlabel("Tamaño de entrenamiento")
    plt.ylabel("RMSE (CV)")
    plt.plot(train_sizes, train_rmse.mean(axis=1), marker='o', label="Entrenamiento")
    plt.plot(train_sizes, test_rmse.mean(axis=1), marker='s', label="Validación CV")
    plt.legend()
    plt.show()


## Generación y limpieza de datos

In [ ]:
from sklearn.datasets import make_regression
X_syn, y_syn = make_regression(n_samples=1200, n_features=8, n_informative=6, noise=10.0, random_state=42)
X_syn = pd.DataFrame(X_syn, columns=[f"x{i}" for i in range(8)])
y_syn = pd.Series(y_syn, name="target")

# Añadir no linealidades e interacciones
X_syn["x_sin"] = np.sin(X_syn["x0"]) * 10
X_syn["x_interact"] = X_syn["x1"] * X_syn["x2"]

# Outliers en ~1.5%
n_out = int(0.015 * len(X_syn))
rows_out = np.random.choice(X_syn.index, size=n_out, replace=False)
X_syn.loc[rows_out, "x3"] *= 10
y_syn.loc[rows_out] *= 5

# Faltantes en ~2% de algunas columnas
for c in ["x0", "x4", "x_interact"]:
    mask = np.random.rand(len(X_syn)) < 0.02
    X_syn.loc[mask, c] = np.nan

print("Forma de X_synthetic:", X_syn.shape)
print("Faltantes por columna:\n", X_syn.isna().sum())

# Capado IQR (usamos una imputación temporal de medianas para poder capar sin NaNs)
X_syn_capped = cap_outliers_iqr(X_syn.fillna(X_syn.median(numeric_only=True)), X_syn.columns)

Xtr, Xte, ytr, yte = train_test_split(X_syn_capped, y_syn, test_size=0.2, random_state=42)

num_features_syn = list(Xtr.columns)
preprocess_syn = ColumnTransformer(
    transformers=[("num", SimpleImputer(strategy="median"), num_features_syn)]
)


## Modelo base

In [ ]:
pipe_syn = Pipeline([
    ("prep", preprocess_syn),
    ("model", DecisionTreeRegressor(random_state=42))
])

pipe_syn.fit(Xtr, ytr)
pred_tr_syn = pipe_syn.predict(Xtr)
pred_te_syn = pipe_syn.predict(Xte)

print("Rendimiento base:")
report_metrics(ytr, pred_tr_syn, label="Train ")
report_metrics(yte, pred_te_syn, label="Test  ")


## Curva de aprendizaje

In [ ]:
plot_learning_curve(DecisionTreeRegressor(random_state=42), Xtr, ytr, title="Curva de aprendizaje — Árbol base (Sintético)")


## Tuning con GridSearchCV

In [ ]:
param_grid_syn = {
    "model__max_depth": [None, 4, 6, 10],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 5],
    "model__ccp_alpha": [0.0, 0.0005, 0.001, 0.01]
}
grid_syn = GridSearchCV(pipe_syn, param_grid_syn, cv=5, scoring="neg_mean_squared_error")
grid_syn.fit(Xtr, ytr)
print("Mejores parámetros (sintético):", grid_syn.best_params_)

best_syn = grid_syn.best_estimator_
pred_tr_best = best_syn.predict(Xtr)
pred_te_best = best_syn.predict(Xte)
print("\nRendimiento con tuning:")
report_metrics(ytr, pred_tr_best, label="Train ")
report_metrics(yte, pred_te_best, label="Test  ")


## Importancias y árbol truncado

In [ ]:
final_tree_syn = best_syn.named_steps["model"]
importances_syn = final_tree_syn.feature_importances_
plt.figure()
order_syn = np.argsort(importances_syn)[::-1]
plt.bar(range(len(importances_syn)), importances_syn[order_syn])
plt.xticks(range(len(importances_syn)), np.array(num_features_syn)[order_syn], rotation=45, ha='right')
plt.title("Importancia de características — Sintético")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plot_tree(final_tree_syn, max_depth=3, filled=True, feature_names=num_features_syn)
plt.title("Árbol de decisión (truncado) — Sintético")
plt.show()


## Conclusiones
- El árbol maneja bien relaciones no lineales y outliers con el flujo de limpieza.
- El tuning mejora la generalización.
- Puedes probar ensambles (RandomForest, Gradient Boosting) para mayor robustez.


## 🧠 Interpretación automática del modelo
Esta celda resume el desempeño del modelo con un texto interpretativo y detecta posibles signos de sobreajuste o subajuste.


In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Predicciones del mejor modelo
y_pred_tr_best = best_syn.predict(Xtr)
y_pred_te_best = best_syn.predict(Xte)

# Métricas
mae_tr = mean_absolute_error(ytr, y_pred_tr_best)
mse_tr = mean_squared_error(ytr, y_pred_tr_best)
rmse_tr = math.sqrt(mse_tr)
r2_tr = r2_score(ytr, y_pred_tr_best)

mae_te = mean_absolute_error(yte, y_pred_te_best)
mse_te = mean_squared_error(yte, y_pred_te_best)
rmse_te = math.sqrt(mse_te)
r2_te = r2_score(yte, y_pred_te_best)

print("Resumen de métricas (mejor modelo):")
print(f"Train -> MAE: {mae_tr:.3f} | RMSE: {rmse_tr:.3f} | R²: {r2_tr:.4f}")
print(f"Test  -> MAE: {mae_te:.3f} | RMSE: {rmse_te:.3f} | R²: {r2_te:.4f}\n")

# Interpretación en texto
def nivel(valor, low, mid, high):
    if valor < low: return "bajo"
    if valor < mid: return "medio"
    if valor < high: return "bueno"
    return "muy bueno"

gap = r2_tr - r2_te
nivel_test = nivel(r2_te, 0.2, 0.5, 0.7)

print("🧠 Interpretación:")
print(f"- El modelo explica aproximadamente el {r2_te*100:.1f}% de la variabilidad en el conjunto de prueba (R² {nivel_test}).")
print(f"- El error promedio absoluto (MAE) en prueba es {mae_te:.2f}, y el RMSE es {rmse_te:.2f}.")
if gap > 0.15:
    print(f"- Existe indicio de **sobreajuste** (diferencia Train-Test de R² = {gap:.2f}). Ajusta hiperparámetros (max_depth, min_samples_leaf/split, ccp_alpha) o usa ensambles.")
elif gap < -0.05:
    print(f"- El desempeño en prueba es **mejor** que en entrenamiento (gap R² = {gap:.2f}); revisa la aleatoriedad o la estabilidad con validación cruzada.")
else:
    print(f"- La generalización es **razonable** (gap R² = {gap:.2f}).")
print("- Prueba también RandomForestRegressor o GradientBoostingRegressor para mayor robustez.")


## 📊 Comparativa de métricas — Antes vs. Después del *tuning*
La siguiente tabla resume **MAE**, **RMSE** y **R²** para el modelo base y el mejor modelo tras *GridSearchCV*, en **train** y **test**.


In [ ]:

import math
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 'pipe_syn' entrenado antes del tuning; 'best_syn' es el mejor tras GridSearchCV.
y_tr_base = pipe_syn.predict(Xtr)
y_te_base = pipe_syn.predict(Xte)

y_tr_best = best_syn.predict(Xtr)
y_te_best = best_syn.predict(Xte)

def metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

rows = []
rows.append(["Base", "Train", *metrics(ytr, y_tr_base)])
rows.append(["Base", "Test",  *metrics(yte, y_te_base)])
rows.append(["Tuned", "Train", *metrics(ytr, y_tr_best)])
rows.append(["Tuned", "Test",  *metrics(yte, y_te_best)])

df_comp = pd.DataFrame(rows, columns=["Modelo","Split","MAE","RMSE","R2"])
display(df_comp.round(4))
